In [ ]:
import asyncio
import websockets
from PIL import Image
from transformers import pipeline

# Load the NSFW classification model
classifier = pipeline("image-classification", model="Falconsai/nsfw_image_detection")

async def handle_connection(websocket, path):
    while True:
        # Receive message from the client (video frame)
        message = await websocket.recv()
        data = json.loads(message)
        if data['type'] == 'frame':
            # Classify the received frame
            frame = Image.open(io.BytesIO(base64.b64decode(data['data'])))
            result = classifier(frame)
            max_label = max(result, key=lambda x: x['score'])
            classification = max_label['label']

            # Send the classification result back to the client
            await websocket.send(json.dumps({
                'type': 'classification',
                'result': classification
            }))

start_server = websockets.serve(handle_connection, 'localhost', 8080)
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()
